In [9]:
import import_ipynb
import oss_handler
from oss_handler import OssClient
import sys, os
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from pylab import mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime

import warnings
warnings.filterwarnings('ignore')

# 数据

In [10]:
oss_client = OssClient()

In [11]:
idx000001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000001.XSHG_hist_index.pkl') #上证指数
idx000016 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000016.XSHG_hist_index.pkl') #上证 50
idx000300 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000300.XSHG_hist_index.pkl') #沪深 300
idx000852 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000852.XSHG_hist_index.pkl') #中证 1000
idx000905 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000905.XSHG_hist_index.pkl') #中证 500
idx000985 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000985.XSHG_hist_index.pkl') #中证全指
# idx399001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399001.XSHG_hist_index.pkl') #深证成指
# idx399006 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399006.XSHG_hist_index.pkl') #创业板指
# idx399303 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399303.XSHG_hist_index.pkl') #国证 2000

In [12]:
adj_close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_close.pk')
adj_factor = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_factor.pk')
adj_high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_high.pk')
adj_low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_low.pk')
adj_open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_open.pk')

circulation_a = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/circulation_a.pk') #流通股本
total_a = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/total_a.pk') #总股本
circulation_market_value = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/circulation_market_value.pk') #流通市值

volume = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/volume.pk') #成交量
num_trades = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/num_trades.pk') #交易次数
total_turnover = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/total_turnover.pk') #换手额
turnover_rate = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/turnover_rate.pk') #换手率

close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/close.pk') 
open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/open.pk') 
high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/high.pk') 
low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/low.pk') 

limit_down = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_down.pk') #跌停
limit_up = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_up.pk') #涨停

halt_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/halt_status.pk') #停牌 
st_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/st_status.pk') #st

In [13]:
BS_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/BS_pit.pkl')
CF_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/CF_pit.pkl')
IS_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/IS_pit.pkl')
all_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/all_pit_fund_new_api.pkl')

# current_perf = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/current_perf.pkl') #当下表现
# perf_forecast = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/perf_forecast.pkl') #预期
# devidend = oss_client.read_oss_parquet_file('ad_hoc_prod/pit_fundemental/rq_dividend.parquet') 

In [14]:
##正常交易非st，halt的票打1.其他为np.nan
st_status = st_status.replace(True,np.nan).replace(False,True)
halt_status = halt_status.replace(True,np.nan).replace(False,True)

adj_return =  adj_close/adj_close.shift(1) - 1
adj_return = adj_return.replace(np.inf,np.nan).replace(-np.inf,np.nan)

##调仓周期
o1o2 = adj_open.shift(-2)/adj_open.shift(-1) - 1 #1日调仓
# o1o2 = adj_open.shift(-5) / adj_open.shift(-1) - 1 #5日调仓
# o1o2 = adj_open.shift(-10) / adj_open.shift(-1) - 1 #10日调仓
# o1o2 = adj_open.shift(-20) / adj_open.shift(-1) - 1 #20日调仓

mc = total_a * adj_close

csi500 = idx000905[['open','close','low']].set_index(idx000905['date'])
idx_500_ret = (csi500['close']/csi500['close'].shift(1) - 1)

mclose = csi500['close']
mopen = csi500['open']
mlow = csi500['low']

open['label'] = 1

# 中性化

In [15]:
##因子中性化 需要的数据
risk1 = pd.read_pickle('D:/redata/risk_factor/logcmv.pk')
risk2 = pd.read_pickle('D:/redata/risk_factor/retsum120.pk')
ind = pd.read_pickle('D:/redata/risk_factor/ind_label.pk')
risk = risk1.merge(risk2,on=['date','ticker'],how='left')
risk = risk.merge(ind,on=['date','ticker'],how='left')
risk = risk[(risk['date']>'2017-01-01')]

In [16]:
##行业中性 2023.8.3 更新
def neturalize_ind(factor,risk=risk,risk_cols=['logcmv','retsum120']):
    XY = pd.merge(risk, factor, on=['date', 'ticker'], how='right')
    #print(XY.head(3))
    XY = XY.dropna()
    tradeday = XY.date.unique()
    daily_resid = []
    for date in tqdm(tradeday[:]):
        xy = XY[XY['date'] == date]
        #只保留有因子值的数据
        xy = xy.dropna()

        #数据标准化
        xy[risk_cols] = (xy[risk_cols] - xy[risk_cols].mean())/xy[risk_cols].std()                
        xy['factor'] = (xy['factor']-xy['factor'].mean())/xy['factor'].std() 

        #选出x y
        x = xy.iloc[:,2:-1]
        y = xy['factor']
        #回归
        multi_linear = LinearRegression(fit_intercept=False)  #无截距项
        multi_linear.fit(x, y)
        beta = multi_linear.coef_
        
        #取残差
        xy['beta'] = np.dot(x, beta)
        xy['resid'] = xy['factor'] - xy['beta']
        daily_resid.append(xy)
    data = pd.concat(daily_resid)
    factor = data[['date', 'ticker', 'factor', 'resid']]
    return factor

In [17]:
def factor_neutralize(df,factor_type,if_rank=True,risk=None,risk_cols=None): #factor_type=hist原值；ind行业及其他中性；单个风险因子文件名； if_rank：中性化之前时候对因子做rank处理
    #中性之前是否对因子做rank处理
    if if_rank == True:
        df = df.rank(axis=1, ascending=True)  # 原值越大rank值越大   
    #因子原值
    if factor_type == 'hist':
        factor_rank = df.copy()
    #行业市值中性
    elif factor_type == 'ind':  ###行业和风格中性
        def neu_process(df):
            df = df.stack().reset_index()
            df.columns = ['date','ticker','factor']
            df = neturalize_ind(df,risk,risk_cols)
            resid = df.set_index(['date','ticker'])['resid'].unstack()
            return resid        
        factor_rank = neu_process(df)
        factor_rank = factor_rank.rank(axis=1,ascending=True) 
        
    return factor_rank

In [18]:
def neu_process(df):
    df = df.stack().reset_index()
    df.columns = ['date','ticker','factor']
    df = neturalize_ind(df,risk=risk,risk_cols=['logcmv','retsum120'])
    resid = df.set_index(['date','ticker'])['resid'].unstack()
    return resid        

In [19]:
def select_num_stocks(n):
    select_stocks = (factor_rank <= n)
    select_stocks_last = select_stocks.iloc[-1:,]
    select_stocks_last = select_stocks_last.replace(True,1).replace(False,np.nan)
    select_stocks_last = select_stocks_last.dropna(how='all',axis=1)
    return select_stocks_last

In [20]:
##市值排uni
mv_rank = risk1.set_index(['date','ticker'])['logcmv'].unstack()
mv_rank = mv_rank.rank(axis=1, ascending=True, pct=True)

##小市值
small = (mv_rank <= 0.33)
small = small.replace(True,1).replace(False,np.nan)
small = small.loc['2017':]

##中市值
median = (mv_rank <= 0.66) & (mv_rank >= 0.33)
median = median.replace(True,1).replace(False,np.nan)
median = median.loc['2017':]

##小中市值
sm = (mv_rank <= 0.66)
sm = sm.replace(True,1).replace(False,np.nan)
sm = sm.loc['2017':]

##大市值
large = (mv_rank>=0.66)
large = large.replace(True,1).replace(False,np.nan)
large = large.loc['2017':]

In [21]:
def position(df, period):
    empty = df > 0
    empty = empty.replace(True,np.nan).replace(False,np.nan)
    
    position = df.iloc[::period]
    position = pd.concat([empty, position])
    position = position.sort_values(['date'])
    position = position.reset_index()
    position = position.drop_duplicates(subset=['date'], keep='last').ffill(limit = period)
    position = position.set_index(['date'])
    return position

In [22]:
path1 = 'Z:/redata/Sen/fundmental_nonfillna' #财报原始数据，单个指标，用于计算因子值的中间数据
path2 = 'Z:/redata/Sen/' #日频财务数据
path_Q = 'Z:/redata/Sen/fundmental_Q'#季度频率，单季度财务数据，用于计算因子值的中间数据

# 财务数据函数

In [23]:
def chg_format(df1):
    df1['ticker'] = df1['order_book_id'].str[:-5]
    df1['date'] = pd.to_datetime(df1['info_date'])
    df1['report_period'] = df1['quarter'].map(lambda x:x.replace('q1','0331').replace('q2','0630').replace('q3','0930').replace('q4','1231'))
    df1['report_period'] = df1['report_period'].astype('int')
    df1 = df1.sort_values(['date','ticker','report_period']).reset_index(drop=True)
    return df1

In [24]:
def func_save(factor,factor_name):
    ##只保留当前最新业绩的数据
    def func(x):
        x = x[x['report_period']>=x['report_period'].expanding(min_periods=1).max()]
        return x
    factor = factor.groupby('ticker').apply(func).reset_index(drop=True)
    #factor同一个date ticker 去重，保留当前最新report_period
    factor = factor.drop_duplicates(subset=['date','ticker'],keep='last')
    factor = factor.set_index(['date','ticker'])[factor_name].unstack()
    factor = pd.concat([factor,Open[['label']]],axis=1).drop(['label'],axis=1)
    factor = factor.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)
    factor.to_pickle(os.path.join(path2,factor_name+'.pkl'))

In [26]:
##利润表流量单季度数据
# dfaa = pd.read_pickle('Z:/fundamental/IS_pit.pkl')
dfaa = IS_pit
dfaa = dfaa.reset_index()
dfaa = chg_format(dfaa)
dfaa = dfaa.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [28]:
#时间表
mls = pd.DataFrame(dfaa.report_period.unique())
mls.columns = ['report_period']
mls['pre_report_period'] = mls['report_period'].shift(1)
mls['pre2_report_period'] = mls['report_period'].shift(2)
mls['pre3_report_period'] = mls['report_period'].shift(3)
mls['next1_report_period'] = mls['report_period'].shift(-1)
mls['next2_report_period'] = mls['report_period'].shift(-2)
mls['next3_report_period'] = mls['report_period'].shift(-3)
mls = mls.replace(np.nan,0)
mls = mls.astype('int')

In [29]:
##现金流量单季度数据
# dfbb = pd.read_pickle('Z:/fundamental/CF_pit.pkl')
dfbb = CF_pit
dfbb = dfbb.reset_index()
dfbb = chg_format(dfbb)
dfbb = dfbb.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [30]:
#时间表
mlss = pd.DataFrame(dfbb.report_period.unique())
mlss.columns = ['report_period']
mlss['pre_report_period'] = mlss['report_period'].shift(1)
mlss['pre2_report_period'] = mlss['report_period'].shift(2)
mlss['pre3_report_period'] = mlss['report_period'].shift(3)
mlss['next1_report_period'] = mlss['report_period'].shift(-1)
mlss['next2_report_period'] = mlss['report_period'].shift(-2)
mlss['next3_report_period'] = mlss['report_period'].shift(-3)
mlss = mlss.replace(np.nan,0)
mlss = mlss.astype('int')

In [31]:
##债务截面单季度数据
# dfcc = pd.read_pickle('Z:/fundamental/BS_pit.pkl')
dfcc = BS_pit
dfcc = dfcc.reset_index()
dfcc = chg_format(dfcc)
dfcc = dfcc.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [32]:
#时间表
mlsss = pd.DataFrame(dfcc.report_period.unique())
mlsss.columns = ['report_period']
mlsss['pre_report_period'] = mlsss['report_period'].shift(1)
mlsss['pre2_report_period'] = mlsss['report_period'].shift(2)
mlsss['pre3_report_period'] = mlsss['report_period'].shift(3)
mlsss['next1_report_period'] = mlsss['report_period'].shift(-1)
mlsss['next2_report_period'] = mlsss['report_period'].shift(-2)
mlsss['next3_report_period'] = mlsss['report_period'].shift(-3)
mlsss = mlsss.replace(np.nan,0)
mlsss = mlsss.astype('int')

In [33]:
#IS,CF的单季度计算
def single_Q(df1,col):
    ##计算单季度数据：原理当季度数据发生更改时，当季的Q和下一季的Q会受到影响。
    ##分别计算当季的Q和下一季的Q 然后两个部分合并到一起。 要保证当季的Q和下一季的Q时，用到的数据日期全部不能大于当前日期。
    #part1：计算当季的Q
    df3 = df1[['date','ticker','quarter','report_period',col]].dropna(subset=[col])
    #拼上对应的前3个季度的report_period
    df3 = df3.merge(mls[['report_period','pre_report_period']],on=['report_period'],how='left')
    df3 = df3.merge(df3,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.sort_values(['date','ticker','report_period','date_pre'])
    #单季数据=当期-上期 ，【第一季用当期】
    df3[col+'_Q'] = df3[col] - df3[col+'_pre']
    df3[col+'_Q'] = np.where(df3['quarter'].str[-2:] == 'q1',df3[col],df3[col+'_Q'])
    #保留所有Q1和除Q1之外date>=date_pre
    df3 = df3[(df3['quarter'].str[-2:]=='q1')|(df3['date']>=df3['date_pre'])] 
    df3 = df3.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #只保留date_pre最新的一条数据
    
    #part2：计算下一季Q
    tmp = df1[['date','ticker','quarter','report_period',col]].dropna(subset=[col])
    #拼上对应的前3个季度的report_period
    tmp = tmp.merge(mls[['report_period','next1_report_period']],on=['report_period'],how='left')
    tmp = tmp.merge(tmp,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    tmp[col+'_Q_next1'] = tmp[col+'_next1'] - tmp[col]
    tmp[col+'_Q_next1'] = np.where(tmp['quarter'].str[-2:] == 'q4',tmp[col+'_next1'],tmp[col+'_Q_next1']) #如果当前是四季度，
    #保留 next_quarter_date<=quarter_date
    tmp = tmp[tmp['date_next1']<=tmp['date']]
    tmp = tmp.drop_duplicates(subset=['date','ticker','report_period'],keep='last')  #只保留next_quarter_date<=quarter_date，且next_quarter_date最新的
    tmp['report_period'] = tmp['report_period_next1']
    tmp = tmp[(tmp['quarter'].str[-2:]!='q4')] #去掉所有当前为4季度的情况，如果当前为4季度，即便数据发成更改，也不影响下一季度的单季数据，因为下一季度是一季度
    tmp[col+'_Q'] = tmp[col+'_Q_next1']
    
    #把两个部分合并就是所有发生更新的数据
    part1 = df3[['date', 'ticker', 'report_period',col+'_Q']]
    part2 = tmp[['date', 'ticker', 'report_period',col+'_Q']]
    # part1['label'] = '1' #可删，只是为了识别数据来源
    # part2['label'] = '2' #可删，只是为了识别数据来源
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [34]:
#同比yoy
def compute_yoy(df4,col):#计算yoy
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_yoy'] = (df1[col]-df1[col+'_pre_yr'])/np.abs(df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_yoy'] = (df2[col+'_next_yr']-df2[col])/np.abs(df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_yoy']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_yoy']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [35]:
#T当季-去年同季
def compute_T(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_T'] = (df1[col]-df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_T'] = (df2[col+'_next_yr']-df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_T']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_T']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [36]:
#Tadd当季+去年同季
def compute_Tadd(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_T'] = (df1[col]+df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_T'] = (df2[col+'_next_yr']+df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_T']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_T']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [37]:
#环比QoQ
def compute_qoq(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date','ticker','report_period','pre_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre'])
    df1 = df1[df1['date']>=df1['date_pre']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_QoQ'] = (df1[col]-df1[col+'_pre'])/np.abs(df1[col+'_pre'])
    part1 = df1[['date','ticker','report_period',col+'_QoQ']]
    #part2
    df1 = df4[['date','ticker','report_period','next1_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next'])
    df1 = df1.sort_values(['date','ticker','report_period','date_next'])
    df1 = df1[df1['date']>=df1['date_next']]
    df1['report_period'] = df1['report_period_next']
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_QoQ'] = (df1[col+'_next']-df1[col])/np.abs(df1[col])
    part2 = df1[['date','ticker','report_period',col+'_QoQ']]

    #把两个部分合并就是所有发生更新的数据
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [38]:
#D 当季-上季
def compute_D(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date','ticker','report_period','pre_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre'])
    df1 = df1[df1['date']>=df1['date_pre']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_D'] = df1[col]-df1[col+'_pre']
    part1 = df1[['date','ticker','report_period',col+'_D']]
    #part2
    df1 = df4[['date','ticker','report_period','next1_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next'])
    df1 = df1.sort_values(['date','ticker','report_period','date_next'])
    df1 = df1[df1['date']>=df1['date_next']]
    df1['report_period'] = df1['report_period_next']
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_D'] = df1[col+'_next']-df1[col]
    part2 = df1[['date','ticker','report_period',col+'_D']]

    #把两个部分合并就是所有发生更新的数据
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [39]:
def compute_ttm(df4,col):
    #单季度数据计算的TTM，季度频率
    tool_data = df4[['date','ticker','report_period',col+'_Q']]
    #part1    
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre3_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_pre3'])]
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_pre3']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_pre3']
    part1 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part1['label'] = 'part1'

    #part2
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_next1'])]
    df3['report_period'] = df3['report_period_next1']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_next1']
    part2 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part2['label'] = 'part2'

    #part3
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])]
    df3['report_period'] = df3['report_period_next2']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_next1','date_next2']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_next1']+ df3[col+'_Q_next2']
    part3 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part3['label'] = 'part3'

    #part4
    df3 = df4.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next3_report_period'],right_on=['ticker','report_period'],suffixes=['','_next3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])&(df3['date']>=df3['date_next3'])]
    df3['report_period'] = df3['report_period_next3']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_next1','date_next1','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_next1'] + df3[col+'_Q_next2']+ df3[col+'_Q_next3']
    part4 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part4['label'] = 'part4'

    part1234 = pd.concat([part1,part2,part3,part4])
    part1234 = part1234.sort_values(['date','ticker','report_period'])
    part1234 = part1234.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part1234

In [40]:
import pandas as pd

def compute_ttm_rate(df4, col):
    # 单季度数据计算的TTM，季度频率
    tool_data = df4[['date', 'ticker', 'report_period', col + '_Q']]
    
    # Part1
    df3 = df4.merge(tool_data, left_on=['ticker', 'pre_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'pre2_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre2'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'pre3_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre3'], how='left')
    df3 = df3[(df3['date'] >= df3['date_pre']) & (df3['date'] >= df3['date_pre2']) & (df3['date'] >= df3['date_pre3'])]
    df3 = df3.sort_values(['ticker', 'date', 'date_pre', 'date_pre2', 'date_pre3']).drop_duplicates(subset=['date', 'ticker', 'report_period'], keep='last')
    df3[col + '_TTM'] = df3[col + '_Q'] + df3[col + '_Q_pre'] + df3[col + '_Q_pre2'] + df3[col + '_Q_pre3']
    part1 = df3[['date', 'ticker', 'report_period', col + '_TTM']]
    
    # Part2
    df3 = df4.merge(tool_data, left_on=['ticker', 'pre_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'pre2_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre2'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'next1_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next1'], how='left')
    df3 = df3[(df3['date'] >= df3['date_pre']) & (df3['date'] >= df3['date_pre2']) & (df3['date'] >= df3['date_next1'])]
    df3['report_period'] = df3['report_period_next1']
    df3 = df3.sort_values(['ticker', 'date', 'date_pre', 'date_pre2', 'date_next1']).drop_duplicates(subset=['date', 'ticker', 'report_period'], keep='last')
    df3[col + '_TTM'] = df3[col + '_Q'] + df3[col + '_Q_pre'] + df3[col + '_Q_pre2'] + df3[col + '_Q_next1']
    part2 = df3[['date', 'ticker', 'report_period', col + '_TTM']]
    
    # Part3
    df3 = df4.merge(tool_data, left_on=['ticker', 'pre_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_pre'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'next1_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next1'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'next2_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next2'], how='left')
    df3 = df3[(df3['date'] >= df3['date_pre']) & (df3['date'] >= df3['date_next1']) & (df3['date'] >= df3['date_next2'])]
    df3['report_period'] = df3['report_period_next2']
    df3 = df3.sort_values(['ticker', 'date', 'date_pre', 'date_next1', 'date_next2']).drop_duplicates(subset=['date', 'ticker', 'report_period'], keep='last')
    df3[col + '_TTM'] = df3[col + '_Q'] + df3[col + '_Q_pre'] + df3[col + '_Q_next1'] + df3[col + '_Q_next2']
    part3 = df3[['date', 'ticker', 'report_period', col + '_TTM']]
    
    # Part4
    df3 = df4.merge(tool_data, left_on=['ticker', 'next1_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next1'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'next2_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next2'], how='left')
    df3 = df3.merge(tool_data, left_on=['ticker', 'next3_report_period'], right_on=['ticker', 'report_period'], suffixes=['', '_next3'], how='left')
    df3 = df3[(df3['date'] >= df3['date_next1']) & (df3['date'] >= df3['date_next2']) & (df3['date'] >= df3['date_next3'])]
    df3['report_period'] = df3['report_period_next3']
    df3 = df3.sort_values(['ticker', 'date', 'date_next1', 'date_next1', 'date_next1']).drop_duplicates(subset=['date', 'ticker', 'report_period'], keep='last')
    df3[col + '_TTM'] = df3[col + '_Q'] + df3[col + '_Q_next1'] + df3[col + '_Q_next2'] + df3[col + '_Q_next3']
    part4 = df3[['date', 'ticker', 'report_period', col + '_TTM']]
    
    part1234 = pd.concat([part1, part2, part3, part4])
    part1234 = part1234.sort_values(['date', 'ticker', 'report_period'])
    part1234 = part1234.drop_duplicates(subset=['date', 'ticker', 'report_period'], keep='last')
    
    # 计算TTM增长率
    part1234['prev_' + col + '_TTM'] = part1234.groupby('ticker')[col + '_TTM'].shift(1)
    part1234[col + '_TTM_growth'] = (part1234[col + '_TTM'] - part1234['prev_' + col + '_TTM']) / part1234['prev_' + col + '_TTM'] * 100
    
    return part1234[['date', 'ticker', 'report_period', col + '_TTM', col + '_TTM_growth']]

# 示例数据
data = {
    'date': pd.date_range(start='2022-03-31', periods=8, freq='Q'),
    'ticker': ['A'] * 8,
    'report_period': pd.date_range(start='2022-03-31', periods=8, freq='Q'),
    'revenue_Q': [100, 110, 120, 130, 140, 150, 160, 170],
    'pre_report_period': pd.date_range(start='2021-12-31', periods=8, freq='Q'),
    'pre2_report_period': pd.date_range(start='2021-09-30', periods=8, freq='Q'),
    'pre3_report_period': pd.date_range(start='2021-06-30', periods=8, freq='Q'),
    'next1_report_period': pd.date_range(start='2022-06-30', periods=8, freq='Q'),
    'next2_report_period': pd.date_range(start='2022-09-30', periods=8, freq='Q'),
    'next3_report_period': pd.date_range(start='2022-12-31', periods=8, freq='Q')
}
df4 = pd.DataFrame(data)

result = compute_ttm_rate(df4, 'revenue')
print(result)

        date ticker report_period  revenue_TTM  revenue_TTM_growth
3 2022-12-31      A    2022-12-31        460.0                 NaN
4 2023-03-31      A    2023-03-31        500.0            8.695652
5 2023-06-30      A    2023-06-30        540.0            8.000000
6 2023-09-30      A    2023-09-30        580.0            7.407407
7 2023-12-31      A    2023-12-31        620.0            6.896552


In [41]:
def process_finfactor_and_save_data(df, col3, Open):
    # 去重
    df_processed = df.drop_duplicates(subset=['date', 'ticker'], keep='last')
    
    # 设置索引并unstack
    df_processed = df_processed.set_index(['date', 'ticker'])[col3].unstack()
    
    # 合并标签数据并删除标签列
    df_processed = pd.concat([df_processed, Open[['label']]], axis=1).drop(['label'], axis=1)
    
    # 填充缺失值并删除全为NaN的行和列
    df_processed = df_processed.fillna(method='ffill', limit=150).dropna(how='all', axis=0).dropna(how='all', axis=1)
    return df_processed

In [42]:
def filter_extreme_by_MAD(series, n=3):
    median = series.median()
    median_new = abs(series - median).median()
    max_value = median + n * median_new
    min_value = median - n * median_new
    return np.clip(series, min_value, max_value, axis=1)

In [43]:
 def func(x):
        x = x[x['report_period']>=x['report_period'].expanding(min_periods=1).max()]
        return x

# Size

In [ ]:
lncap = np.log(circle_mv)
lncap.to_pickle('Z:/redata/Sen/lncap.pkl')

In [ ]:
lncap

In [ ]:
lncap3

In [ ]:
XY = pd.merge(lncap, lncap3, on=['date', 'ticker'], how='right')

In [ ]:
lncap3 = (lncap)**3
lncap3.replace([np.inf, -np.inf], np.nan, inplace=True)

XY = pd.merge(lncap, lncap3, on=['date', 'ticker'], how='right')
    #print(XY.head(3))
XY = XY.dropna()
tradeday = XY.date.unique()
daily_resid = []
for date in tqdm(tradeday[:]):
    xy = XY[XY['date'] == date]
        #只保留有因子值的数据
    xy = xy.dropna() 

        #选出x y
    x = xy.iloc[:,2:-1]
    y = xy['factor']
        #回归
    multi_linear = LinearRegression(fit_intercept=False)  #无截距项
    multi_linear.fit(x, y)
    beta = multi_linear.coef_
        
        #取残差
    xy['beta'] = np.dot(x, beta)
    xy['resid'] = xy['factor'] - xy['beta']
    daily_resid.append(xy)
data = pd.concat(daily_resid)
factor = data[['date', 'ticker', 'factor', 'resid']]
return factor

# BETA因子

In [ ]:
idx_500_ret = pd.DataFrame(idx_500_ret)
idx_500_ret

In [ ]:
excess500 = adj_return - idx_500_ret
excess500.dropna(axis=0)

# 财务因子

In [65]:
#净利润同比增速
#单季度
col1 = 'net_profit'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'net_profit_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'net_profit_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/net_profit_Q_yoy@factor.pkl')

In [66]:
#归母净利润同比增速

#单季度
col1 = 'net_profit_parent_company'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'net_profit_parent_company_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'net_profit_parent_company_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/net_profit_parent_company_Q_yoy@factor.pkl')

In [67]:
# 当季营业同比增长率

#单季度
col1 = 'operating_revenue'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'operating_revenue_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'operating_revenue_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/operating_revenue_Q_yoy@factor.pkl')

In [68]:
# 营业收入ttm增长率

#单季度
col1 = 'operating_revenue'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'operating_revenue'
df_computed = compute_ttm(df_merge,col2)

col3 = 'operating_revenue_TTM'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/operating_revenue_TTM@factor.pkl')

In [69]:
# BP 股东权益/总市值

#单季度
col1 = 'equity_parent_company'
df_singleQ = single_Q(dfcc,col1)

col2 = 'equity_parent_company_Q'
df_fin = process_finfactor_and_save_data(df_singleQ, col2, open)

BP = df_fin / mc
BP.to_pickle('D:/redata/factor/BP@factor.pkl')

In [70]:
# EP 因子

#单季度
col = 'net_profit'
df_single_Q = single_Q(dfaa,col)

col2 = 'net_profit_Q'
df_fin = process_finfactor_and_save_data(df_single_Q, col2, open)

EP = df_fin / mc
EP.to_pickle('D:/redata/factor/EP@factor.pkl')

In [71]:
# CFP_ttm

#单季度
col = 'cash_flow_from_operating_activities'
df_single_Q = single_Q(dfbb,col)
df_merge = df_single_Q.merge(mlss,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
df_ttm = compute_ttm(df_merge,col)

col1 = 'cash_flow_from_operating_activities_TTM'
df_fin = process_finfactor_and_save_data(df_ttm, col1, open)

cfp = df_fin/mc
cfp.to_pickle('Z:/redata/Sen/cfp_ttm@factor.pkl')

In [ ]:
# 增长率(本期-上期）_TTM 先做single_Q
df6= df5.groupby('ticker').apply(func).reset_index(drop=True)
    #factor同一个date ticker 去重，保留当前最新report_period
df6 = df6.drop_duplicates(subset=['date','ticker'],keep='last')
df6 = df6.sort_values(['ticker','report_period'])
df6.set_index(['date','report_period'],inplace = True)
# 计算增长率ttm
df6grouped = df6.groupby('ticker').apply(lambda x:x.pct_change().rolling(4).sum())
df6grouped['ticker'] = df6['ticker']
df6grouped = df6grouped.dropna(how = 'all', axis = 0)
df7 = df6grouped.reset_index()

col = 'operating_revenue_TTM'

df7 = df7.drop_duplicates(subset=['date','ticker'],keep='last')
df7 = df7.set_index(['date','ticker'])[col].unstack()
df7 = pd.concat([df7,Open[['label']]],axis=1).drop(['label'],axis=1)
df7 = df7.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)

df7.to_pickle('Z:/redata/Sen/operating_revenue_rate_TTM_factor.pkl')

In [ ]:
df4

In [ ]:
test = df5.loc[:'2022-10-22']
test

In [ ]:
# 营业外收支利润净额/利润总额
col = 'non_operating_revenue'

#单季度
df1 = single_Q(dfaa,col)

col = 'non_operating_revenue_Q'
df1 = df1.drop_duplicates(subset=['date','ticker'],keep='last')
df1 = df1.set_index(['date','ticker'])[col].unstack()
df1 = pd.concat([df1,Open[['label']]],axis=1).drop(['label'],axis=1)
df1 = df1.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)

col = 'non_operating_expense'

#单季度
df2 = single_Q(dfaa,col)

col = 'non_operating_expense_Q'
df2 = df2.drop_duplicates(subset=['date','ticker'],keep='last')
df2 = df2.set_index(['date','ticker'])[col].unstack()
df2 = pd.concat([df2,Open[['label']]],axis=1).drop(['label'],axis=1)
df2 = df2.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)

col = 'profit_before_tax'

#单季度
df3 = single_Q(dfaa,col)

col = 'profit_before_tax_Q'
df3 = df3.drop_duplicates(subset=['date','ticker'],keep='last')
df3 = df3.set_index(['date','ticker'])[col].unstack()
df3 = pd.concat([df3,Open[['label']]],axis=1).drop(['label'],axis=1)
df3 = df3.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)

df4 = (df1 - df2)/df3
df4 = df4.replace(0, np.nan)
df4 = filter_extreme_by_MAD(df4)

NNOP = -df4
NNOP.to_pickle('Z:/redata/Sen/NNOP_factor.pkl')

# 量价因子

In [ ]:
AOG = (adj_open.shift(-1)/adj_close).sub(mopen.shift(-1) / mclose,axis=0)
AOG.to_pickle('Z:/redata/Sen/AOG_factor.pkl')

In [ ]:
JOR = (adj_low.shift(-1)/adj_close).sub(mlow.shift(-1)/mclose,axis=0)
JOR.to_pickle('Z:/redata/Sen/JOR_factor.pkl')

In [ ]:
OverNight = (adj_open/adj_close.shift(1)).astype('float')
IntraDay = (adj_close/adj_open).astype('float')

ON20 = np.log(OverNight).rolling(20).sum()
IN20 = np.log(IntraDay).rolling(20).sum()

ON20.to_pickle('Z:/redata/Sen/ON20_factor.pkl')
IN20.to_pickle('Z:/redata/Sen/IN20_factor.pkl')

In [ ]:
Mom6m = (1+ adj_return.copy()).rolling(window=125).apply(lambda x: x.prod()-1, raw=True).shift(20)
Mom6m.to_pickle('Z:/redata/Sen/Mom6m_factor.pkl')

In [ ]:
Mom12m = (1+ adj_return.copy()).rolling(window=250).apply(lambda x: x.prod()-1, raw=True).shift(20)
# Mom12m = Mom12m.rolling(20).mean() #平滑处理，换手率>20%
Mom12m.to_pickle('Z:/redata/Sen/Mom12m_factor.pkl')

In [ ]:
Mom36m = (1+ adj_return.copy()).rolling(window=750).apply(lambda x: x.prod()-1, raw=True).shift(250)
# Mom36m = Mom36m.rolling(20).mean() #平滑
Mom36m.to_pickle('Z:/redata/Sen/Mom36m_factor.pkl')

In [ ]:
MaxRet = adj_return.copy().rolling(window=20).max().shift(1)
MaxRet.to_pickle('Z:/redata/Sen/MaxRet_factor.pkl')

In [ ]:
std_turn = turnover_rate.rolling(window=5, closed='left').std().shift(1)
std_turn = std_turn.rolling(10).mean()
std_turn.to_pickle('Z:/redata/Sen/std_turn_factor.pkl')

In [ ]:
VolSD = volume.rolling(window = 750, min_periods=24, closed='left').std()
VolSD.to_pickle('Z:/redata/Sen/VolSD_factor.pkl')

In [ ]:
Momss=adj_return.copy().rolling(window=20).apply(lambda x: (1+x).prod()-1, raw=True).shift(250)
Momss=Momss.rolling(20).mean()
Momss.to_pickle('Z:/redata/Sen/Momss_factor.pkl')

In [ ]:
turnover_std = -(turnover_rate.rolling(window=20).std())

turnover_std = filter_extreme_by_MAD(turnover_std)
turnover_std.to_pickle('Z:/redata/Sen/turnover_std_factor.pkl')

In [ ]:
turnrate_mean = -(turnover_rate.rolling(window=20).mean())
turnrate_mean.to_pickle('Z:/redata/Sen/turnrate_mean_factor.pkl')

In [ ]:
mv = -( volume.rolling(window=20).std())

mv = filter_extreme_by_MAD(mv)
mv.to_pickle('Z:/redata/Sen/mv_factor.pkl')

In [ ]:
alpha95 = -(volume.rolling(5).std())
alpha95.to_pickle('Z:/redata/Sen/alpha95_factor.pkl')

In [ ]:
ret20 = -1*(adj_return.rolling(window=20).sum())
ret20.to_pickle('Z:/redata/Sen/ret_20_factor.pkl')

In [ ]:
retstd20 = -1*(adj_return.rolling(window=20).std())
retstd20.to_pickle('Z:/redata/Sen/retstd20_factor.pkl')

In [ ]:
maketcap = np.log(adj_close * total_amount)
maketcap.to_pickle('Z:/redata/Sen/maketcap_factor.pkl')

In [ ]:
total_amount_5 = total_amount.rolling(5).std() * -1
total_amount_5.to_pickle('Z:/redata/Sen/total_amount_5_factor.pkl')

In [ ]:
def rank(X):
    return X.rank(axis=1, method='min', pct = True)

def delay(X, d):
    return X.shift(d)

def correlation(X, Y, d):
    return X.rolling(window=d).corr(Y.rolling(window=d))

def covariance(X, Y, d):
    return X.rolling(window=d).cov(Y.rolling(window=d))

def ts_max(X, d):
    return X.rolling(window=d).max()

def ts_min(X, d):
    return X.rolling(window=d).min()

alpha3 = -1 * correlation(rank(adj_open), rank(volume), 10)
alpha3.to_pickle('Z:/redata/Sen/alpha3_factor.pkl')

alpha13 = -1 * rank(covariance(rank(adj_close), rank(volume), 20))
# alpha13 = alpha13.rolling(60).mean()
alpha13.to_pickle('Z:/redata/Sen/alpha13_factor.pkl')

alpha15 = -1 * rank(correlation(rank(adj_high), rank(volume), 3)).rolling(window=3).sum()
# alpha15 = alpha15.rolling(60).mean()
alpha15.to_pickle('Z:/redata/Sen/alpha15_factor.pkl')

alpha16 = -1 * rank(covariance(rank(adj_high), rank(volume), 5))
alpha16.to_pickle('Z:/redata/Sen/alpha16_factor.pkl')

alpha44 = -1 * correlation(adj_high, rank(volume), 5)
alpha44.to_pickle('Z:/redata/Sen/alpha44_factor.pkl')

alpha50 = -1 * ts_max(rank(correlation(rank(volume), rank(adj_close), 5)), 5)
alpha50.to_pickle('Z:/redata/Sen/alpha50_factor.pkl')

close_normalized = (adj_close - adj_low.rolling(12).min()) / (adj_high.rolling(12).max()- adj_low.rolling(12).min())
alpha55 = -1 * correlation(rank(close_normalized), rank(volume), 6)
alpha55.to_pickle('Z:/redata/Sen/alpha55_factor.pkl')

Var120 = -(adj_return.rolling(window = 121).var())
Var120 = filter_extreme_by_MAD(Var120)
Var120.to_pickle('Z:/redata/Sen/Var120_factor.pkl')

In [ ]:
liquidity = -1 * (0.25 * turnover_rate.rolling(20).mean() + 0.35 * turnover_rate.rolling(60).mean() + 0.3 * turnover_rate.rolling(252).mean())
liquidity.to_pickle('Z:/redata/Sen/liquidity_factor.pkl')

In [ ]:
#买卖压力撮合因子

vwap = total_turnover / volume

def apb(T):
    a = vwap.rolling(window=T).mean()
    b = (volume * vwap).rolling(window=T).sum()
    c = volume.rolling(window=T).sum()
    apb = np.log((a)/(b/c))
    return apb

APB_5 = apb(T=5)
APB_5 = APB_5.rolling(20).mean()
APB_5 = filter_extreme_by_MAD(APB_5)
APB_5.to_pickle('Z:/redata/Sen/APB_5_factor.pkl')

APB_20 = (apb(T=20)).rolling(20).mean()
APB_20 = filter_extreme_by_MAD(APB_20)
APB_20.to_pickle('Z:/redata/Sen/APB_20_factor.pkl')

In [ ]:
vwap = total_turnover / volume
n_days = 20
a = vwap.rolling(window=n_days,min_periods=int(n_days/2)).mean()
b = (volume * vwap).rolling(window=n_days,min_periods=int(n_days/2)).mean()
c = volume.rolling(window=n_days,min_periods=int(n_days/2)).mean()
APB_1 = np.log((a/(b/c)))
APB_1 = APB_1.rolling(20).mean()
APB_1.to_pickle('Z:/redata/Sen/APB_1_factor.pkl')

In [ ]:
wt = 0.5 ** (1/42)
daily_standard_deviation = np.sqrt(wt * ((adj_return - adj_return.rolling(252).mean())**2).rolling(252).sum())
daily_standard_deviation.to_pickle('Z:/redata/Sen/daily_standard_deviation_factor.pkl')

In [ ]:
high1 = adj_high
low1 = adj_low
close1 = adj_close

high_r_std_Nm = (high1 / close1.shift(1)).rolling(120).std()
low_r_std_Nm = (low1 / close1.shift(1)).rolling(120).std()

hml_r_std_Nm = (high_r_std_Nm - low_r_std_Nm) * -1
hpl_r_std_Nm = (high_r_std_Nm + low_r_std_Nm) * -1

hml_r_std_Nm = filter_extreme_by_MAD(hml_r_std_Nm)
hpl_r_std_Nm = filter_extreme_by_MAD(hpl_r_std_Nm)

hml_r_std_Nm.to_pickle('Z:/redata/Sen/hml_r_std_6m_factor.pkl')
hpl_r_std_Nm.to_pickle('Z:/redata/Sen/hpl_r_std_6m_factor.pkl')

high_r_std_N = (high1 / close1.shift(1)).rolling(100).std()
high_r_std_N = -1 * high_r_std_Nm
high_r_std_N = filter_extreme_by_MAD(high_r_std_N)
high_r_std_N.to_pickle('Z:/redata/Sen/high_r_std_5m_factor.pkl')

In [ ]:
total_marketcap = -mc
total_marketcap.to_pickle('Z:/redata/Sen/total_marketcap_factor.pkl')

In [ ]:
cir_mv = -circle_mv
cir_mv.to_pickle('Z:/redata/Sen/cir_mv_factor.pkl')

In [ ]:
ma_5 = adj_close.rolling(5).mean() * -1
bias_5 = (adj_close/ma_5) - 1
bias_5.to_pickle('Z:/redata/Sen/bias_5_factor.pkl')

In [ ]:
adjclose = adj_close * -1
adjclose.to_pickle('Z:/redata/Sen/adjclose_factor.pkl')

In [ ]:
uppershadow = adj_high - np.maximum(adj_open, adj_close)
lowershadow = np.minimum(adj_open, adj_close) - adj_low
_stduppershadow = uppershadow / uppershadow.rolling(5).mean().shift(1)
_stdlowershadow = lowershadow / lowershadow.rolling(5).mean().shift(1)

william_uppershadow = adj_high - adj_close
william_lowershadow = adj_close - adj_low
_stdwilliam_uppershadow = william_uppershadow / william_uppershadow.rolling(5).mean().shift(1)
_stdwilliam_lowershadow = william_lowershadow / william_lowershadow.rolling(5).mean().shift(1)

uppershadow_std = _stduppershadow.rolling(20).std()
william_lowershadow_mean = _stdwilliam_lowershadow.rolling(20).mean()

uppershadow_std.to_pickle('Z:/redata/Sen/uppershadow_std_factor.pkl')
william_lowershadow_mean.to_pickle('Z:/redata/Sen/william_lowershadow_mean_factor.pkl')

In [ ]:
# 采用等权方式构造UBL因子
UBL = (uppershadow_std + william_lowershadow_mean)*-1
UBL = UBL.rolling(20).mean()
UBL.to_pickle('Z:/redata/Sen/UBL_factor.pkl')

# 行业市值中性化

In [68]:
def compute_condition(new_stk=20):
    drop_new_stk = adj_open.rolling(window=new_stk, min_periods=new_stk).mean()
    drop_new_stk = ~np.isnan(drop_new_stk)
    df = st_status * halt_status * drop_new_stk
    df = df.replace(True, 1).replace(False, np.nan)
    return df.loc['2017':]

In [69]:
def process_factor(factor_path, output_path, final_universe, risk, risk_cols):
    df = pd.read_pickle(factor_path)
    df = df.replace([np.inf, -np.inf], np.nan)
    df_clean = df * final_universe
    df_clean = df_clean.dropna(how='all', axis=0)
    factor_type = 'ind'
    factor_rank = factor_neutralize(df_clean, factor_type, if_rank=True, risk=risk, risk_cols=risk_cols)
    factor_rank.to_pickle(output_path)

# Assuming final_universe and risk are defined elsewhere in your code
final_universe = compute_condition(new_stk = 10) # Define your final_universe DataFrame
risk = risk  # Define your risk DataFrame
risk_cols = ['logcmv', 'retsum120']

# List of factors and their corresponding paths
factors = [
    ('Z:/Sen/cfp_ttm_factor.pkl', 'Z:/Sen/Neutraliza/cfp_ttm_nu.pkl'),
    ('Z:/Sen/EP_factor.pkl', 'Z:/Sen/Neutraliza/EP_nu.pkl'),
    ('Z:/Sen/net_profit_Q_yoy_factor.pkl', 'Z:/Sen/Neutraliza/net_profit_Q_yoy_nu.pkl'),
    ('Z:/Sen/turnover_std_factor.pkl', 'Z:/Sen/Neutraliza/turnover_std_nu.pkl'),
    ('Z:/Sen/mv_factor.pkl', 'Z:/Sen/Neutraliza/mv_nu.pkl'),
    ('Z:/Sen/Var120_factor.pkl', 'Z:/Sen/Neutraliza/Var120_nu.pkl'),
    ('Z:/Sen/hml_r_std_6m_factor.pkl', 'Z:/Sen/Neutraliza/hml_r_std_6m_nu.pkl'),
    ('Z:/Sen/liquidity_factor.pkl', 'Z:/Sen/Neutraliza/liquidity_nu.pkl'),
    ('Z:/Sen/APB_20_factor.pkl', 'Z:/Sen/Neutraliza/APB_20_nu.pkl'),
    ('Z:/Sen/high_r_std_5m_factor.pkl','Z:/Sen/Neutraliza/high_r_std_5m_nu.pkl'),
    ('Z:/Sen/hpl_r_std_6m_factor.pkl','Z:/Sen/Neutraliza/hpl_r_std_6m_nu.pkl'),
    ('Z:/Sen/BP_factor.pkl','Z:/Sen/Neutraliza/BP_nu.pkl'),
    ('Z:/Sen/total_marketcap_factor.pkl','Z:/Sen/Neutraliza/total_marketcap_nu.pkl'),
    ('Z:/Sen/cir_mv_factor.pkl','Z:/Sen/Neutraliza/cir_mv_nu.pkl'),
    ('Z:/Sen/ret_20_factor.pkl','Z:/Sen/Neutraliza/ret_20_nu.pkl'),
    ('Z:/Sen/bias_5_factor.pkl','Z:/Sen/Neutraliza/bias_5_nu.pkl'),
    ('Z:/Sen/total_amount_5_factor.pkl','Z:/Sen/Neutraliza/total_amount_5_nu.pkl'),
    ('Z:/Sen/adjclose_factor.pkl','Z:/Sen/Neutraliza/adjclose_nu.pkl')
]

# Process each factor
for factor_path, output_path in factors:
    process_factor(factor_path, output_path, final_universe, risk, risk_cols)

100%|██████████████████████████████████████████████████████████████████████████████| 1616/1616 [01:16<00:00, 21.19it/s]


In [141]:
def compute_IC(factor_rank, ret_rank):
    IC = (factor_rank.corrwith(ret_rank, axis=1)).dropna()
    IC_mean = IC.mean()
    IC_std = IC.std()
    ICIR = IC_mean / IC_std
    ICres = pd.DataFrame([[IC_mean, ICIR]], columns=['ICmean', 'ICIR'])
    ICIR_120_weight = IC.rolling(120).apply(lambda x: x.mean() / x.std()).dropna()
    return IC, ICres, ICIR_120_weight

# Assuming factor_rank and ret_rank are defined elsewhere in your code
# factor_rank = ... # Define your factor_rank DataFrame
ret_rank = o1o2.rank(axis=1, ascending=True)

factors_nu = [
('D:/redata/Neutraliza/cfp_ttm_nu.pkl', 'D:/redata/ICIR/cfp_ttm_ICIR.pkl'),
('D:/redata/Neutraliza/EP_nu.pkl', 'D:/redata/ICIR/EP_ICIR.pkl'),
('D:/redata/Neutraliza/net_profit_Q_yoy_nu.pkl', 'D:/redata/ICIR/net_profit_Q_yoy_ICIR.pkl'),
('D:/redata/Neutraliza/turnover_std_nu.pkl', 'D:/redata/ICIR/turnover_std_ICIR.pkl'),
('D:/redata/Neutraliza/mv_nu.pkl', 'D:/redata/ICIR/mv_ICIR.pkl'),
('D:/redata/Neutraliza/Var120_nu.pkl', 'D:/redata/ICIR/Var120_ICIR.pkl'),
('D:/redata/Neutraliza/hml_r_std_6m_nu.pkl', 'D:/redata/ICIR/hml_r_std_6m_ICIR.pkl'),
('D:/redata/Neutraliza/liquidity_nu.pkl', 'D:/redata/ICIR/liquidity_ICIR.pkl'),
('D:/redata/Neutraliza/APB_20_nu.pkl', 'D:/redata/ICIR/APB_20_ICIR.pkl'), 
('D:/redata/Neutraliza/high_r_std_5m_nu.pkl','D:/redata/ICIR/high_r_std_5m_ICIR.pkl'),
('D:/redata/Neutraliza/hpl_r_std_6m_nu.pkl','D:/redata/ICIR/hpl_r_std_6m_ICIR.pkl'),
('D:/redata/Neutraliza/BP_nu.pkl','D:/redata/ICIR/BP_ICIR.pkl'),
('D:/redata/Neutraliza/total_marketcap_nu.pkl','D:/redata/ICIR/total_marketcap_ICIR.pkl'),
('D:/redata/Neutraliza/cir_mv_nu.pkl','D:/redata/ICIR/cir_mv_ICIR.pkl'),
('D:/redata/Neutraliza/ret_20_nu.pkl','D:/redata/ICIR/ret_20_ICIR.pkl'),
('D:/redata/Neutraliza/bias_5_nu.pkl','D:/redata/ICIR/bias_5_ICIR.pkl'),
('D:/redata/Neutraliza/total_amount_5_nu.pkl','D:/redata/ICIR/total_amount_5_ICIR.pkl'),
('D:/redata/Neutraliza/adjclose_nu.pkl','D:/redata/ICIR/adjclose_ICIR.pkl')
]

# Process each factor
for factor_path, output_path in factors_nu:
    factor_rank = pd.read_pickle(factor_path)
# Assuming ret_rank is defined and available
    IC, ICres, ICIR_120_weight = compute_IC(factor_rank, ret_rank)
# Optionally save ICIR_120_weight if needed
    ICIR_120_weight.to_pickle(output_path.replace('.pkl', '_120_weight.pkl'))

In [142]:
import pandas as pd

# List of file paths
f_files = [
    'D:/redata/Neutraliza/cfp_ttm_nu.pkl',
    'D:/redata/Neutraliza/EP_nu.pkl',
    'D:/redata/Neutraliza/net_profit_Q_yoy_nu.pkl',
    'D:/redata/Neutraliza/turnover_std_nu.pkl',
    'D:/redata/Neutraliza/mv_nu.pkl',
    'D:/redata/Neutraliza/Var120_nu.pkl',
    'D:/redata/Neutraliza/hml_r_std_6m_nu.pkl',
    'D:/redata/Neutraliza/liquidity_nu.pkl',
    'D:/redata/Neutraliza/APB_20_nu.pkl',
    'D:/redata/Neutraliza/high_r_std_5m_nu.pkl',
    'D:/redata/Neutraliza/hpl_r_std_6m_nu.pkl',
    'D:/redata/Neutraliza/BP_nu.pkl',
    'D:/redata/Neutraliza/total_marketcap_nu.pkl',
    'D:/redata/Neutraliza/cir_mv_nu.pkl',
    'D:/redata/Neutraliza/ret_20_nu.pkl',
    'D:/redata/Neutraliza/bias_5_nu.pkl',
    'D:/redata/Neutraliza/total_amount_5_nu.pkl',
    'D:/redata/Neutraliza/adjclose_nu.pkl'
]

i_files = [
    'D:/redata/ICIR/cfp_ttm_ICIR_120_weight.pkl',
    'D:/redata/ICIR/EP_ICIR_120_weight.pkl',
    'D:/redata/ICIR/net_profit_Q_yoy_ICIR_120_weight.pkl',
    'D:/redata/ICIR/turnover_std_ICIR_120_weight.pkl',
    'D:/redata/ICIR/mv_ICIR_120_weight.pkl',
    'D:/redata/ICIR/Var120_ICIR_120_weight.pkl',
    'D:/redata/ICIR/hml_r_std_6m_ICIR_120_weight.pkl',
    'D:/redata/ICIR/liquidity_ICIR_120_weight.pkl',
    'D:/redata/ICIR/APB_20_ICIR_120_weight.pkl',
    'D:/redata/ICIR/high_r_std_5m_ICIR_120_weight.pkl',
    'D:/redata/ICIR/hpl_r_std_6m_ICIR_120_weight.pkl',
    'D:/redata/ICIR/BP_ICIR_120_weight.pkl',
    'D:/redata/ICIR/total_marketcap_ICIR_120_weight.pkl',
    'D:/redata/ICIR/cir_mv_ICIR_120_weight.pkl',
    'D:/redata/ICIR/ret_20_ICIR_120_weight.pkl',
    'D:/redata/ICIR/bias_5_ICIR_120_weight.pkl',
    'D:/redata/ICIR/total_amount_5_ICIR_120_weight.pkl',
    'D:/redata/ICIR/adjclose_ICIR_120_weight.pkl'
]

o_files = [
    'D:/redata/CombineFactors/cfp_ttm_ICIR_combine.pkl',
    'D:/redata/CombineFactors/EP_ICIR_combine.pkl',
    'D:/redata/CombineFactors/net_profit_Q_yoy_ICIR_combine.pkl',
    'D:/redata/CombineFactors/turnover_std_ICIR_combine.pkl',
    'D:/redata/CombineFactors/mv_ICIR_combine.pkl',
    'D:/redata/CombineFactors/Var120_ICIR_combine.pkl',
    'D:/redata/CombineFactors/hml_r_std_6m_ICIR_combine.pkl',
    'D:/redata/CombineFactors/liquidity_ICIR_combine.pkl',
    'D:/redata/CombineFactors/APB_20_ICIR_combine.pkl',
    'D:/redata/CombineFactors/high_r_std_5m_ICIR_combine.pkl',
    'D:/redata/CombineFactors/hpl_r_std_6m_ICIR_combine.pkl',
    'D:/redata/CombineFactors/BP_ICIR_combine.pkl',
    'D:/redata/CombineFactors/total_marketcap_ICIR_combine.pkl',
    'D:/redata/CombineFactors/cir_mv_ICIR_combine.pkl',
    'D:/redata/CombineFactors/ret_20_ICIR_combine.pkl',
    'D:/redata/CombineFactors/bias_5_ICIR_combine.pkl',
    'D:/redata/CombineFactors/total_amount_5_ICIR_combine.pkl',
    'D:/redata/CombineFactors/adjclose_ICIR_combine.pkl'
]

# Read f files
f_dataframes = [pd.read_pickle(file) for file in f_files]

# Read i files and convert to DataFrame
i_dataframes = []
for file in i_files:
    df = pd.read_pickle(file)
    df = pd.DataFrame(df)
    df.columns = ['ICIR']
    i_dataframes.append(df)

# Perform multiplication and save results
results = []
for f_df, i_df, o_file in zip(f_dataframes, i_dataframes, o_files):
    result = f_df.mul(i_df['ICIR'], axis=0)
    result.to_pickle(o_file)
    results.append(result)

# Now results list contains all multiplication operation后的 DataFrame

In [143]:
f1 = pd.read_pickle( 'D:/redata/CombineFactors/cfp_ttm_ICIR_combine.pkl')
f2 = pd.read_pickle( 'D:/redata/CombineFactors/EP_ICIR_combine.pkl')
f3 = pd.read_pickle( 'D:/redata/CombineFactors/net_profit_Q_yoy_ICIR_combine.pkl')
f4 = pd.read_pickle( 'D:/redata/CombineFactors/turnover_std_ICIR_combine.pkl')
f5 = pd.read_pickle( 'D:/redata/CombineFactors/mv_ICIR_combine.pkl')
f6 = pd.read_pickle( 'D:/redata/CombineFactors/Var120_ICIR_combine.pkl')
f7 = pd.read_pickle( 'D:/redata/CombineFactors/hml_r_std_6m_ICIR_combine.pkl')
f8 = pd.read_pickle( 'D:/redata/CombineFactors/liquidity_ICIR_combine.pkl')
f9 = pd.read_pickle( 'D:/redata/CombineFactors/APB_20_ICIR_combine.pkl')
f10 = pd.read_pickle( 'D:/redata/CombineFactors/high_r_std_5m_ICIR_combine.pkl')
f11 = pd.read_pickle( 'D:/redata/CombineFactors/hpl_r_std_6m_ICIR_combine.pkl')
f12 = pd.read_pickle( 'D:/redata/CombineFactors/BP_ICIR_combine.pkl')
f13 = pd.read_pickle( 'D:/redata/CombineFactors/total_marketcap_ICIR_combine.pkl')
f14 = pd.read_pickle( 'D:/redata/CombineFactors/cir_mv_ICIR_combine.pkl')
f15 = pd.read_pickle( 'D:/redata/CombineFactors/ret_20_ICIR_combine.pkl')
f16 = pd.read_pickle( 'D:/redata/CombineFactors/bias_5_ICIR_combine.pkl')
f17 = pd.read_pickle( 'D:/redata/CombineFactors/total_amount_5_ICIR_combine.pkl')
f18 = pd.read_pickle( 'D:/redata/CombineFactors/adjclose_ICIR_combine.pkl')

In [ ]:
def factor_neutralize(df, factor_type, if_rank=True, risk=None, risk_cols=None):
    if if_rank:
        df = df.rank(axis=1, ascending=True)  # 原值越大rank值越大
    if factor_type == 'hist':
        factor_rank = df.copy()
    elif factor_type == 'ind':
        def neu_process(df):
            df = df.stack().reset_index()
            df.columns = ['date', 'ticker', 'factor']
            df = neturalize_ind(df, risk, risk_cols)
            resid = df.set_index(['date', 'ticker'])['resid'].unstack()
            return resid
        factor_rank = neu_process(df)
        factor_rank = factor_rank.rank(axis=1, ascending=True)
    return factor_rank

def compute_condition(new_stk=20):
    drop_new_stk = adj_open.rolling(window=new_stk, min_periods=new_stk).mean()
    drop_new_stk = ~np.isnan(drop_new_stk)
    df = st_status * halt_status * drop_new_stk
    df = df.replace(True, 1).replace(False, np.nan)
    return df.loc['2017':]

def compute_IC(factor_rank, ret_rank):
    IC = (factor_rank.corrwith(ret_rank, axis=1)).dropna()
    IC_mean = IC.mean()
    IC_std = IC.std()
    ICIR = IC_mean / IC_std
    ICres = pd.DataFrame([[IC_mean, ICIR]], columns=['ICmean', 'ICIR'])
    ICIR_120_weight = IC.rolling(120).apply(lambda x: x.mean() / x.std()).dropna()
    return IC, ICres, ICIR_120_weight

final_universe = compute_condition(new_stk=10)
risk = risk
risk_cols = ['logcmv', 'retsum120']

factors = [
    ('D:/redata/factor/cfp_ttm_factor.pkl', 'D:/redata/Neutraliza/cfp_ttm_nu.pkl'),
    ('D:/redata/factor/EP_factor.pkl', 'D:/redata/Neutraliza/EP_nu.pkl'),
    ('D:/redata/factor/net_profit_Q_yoy_factor.pkl', 'D:/redata/Neutraliza/net_profit_Q_yoy_nu.pkl'),
    ('D:/redata/factor/turnover_std_factor.pkl', 'Z:/redata/Neutraliza/turnover_std_nu.pkl'),
    ('D:/redata/factor/mv_factor.pkl', 'D:/redata/Neutraliza/mv_nu.pkl'),
    ('D:/redata/factor/Var120_factor.pkl', 'D:/redata/Neutraliza/Var120_nu.pkl'),
    ('D:/redata/factor/hml_r_std_6m_factor.pkl', 'D:/redata/Neutraliza/hml_r_std_6m_nu.pkl'),
    ('D:/redata/factor/liquidity_factor.pkl', 'D:/redata/Neutraliza/liquidity_nu.pkl'),
    ('D:/redata/factor/APB_20_factor.pkl', 'D:/redata/Neutraliza/APB_20_nu.pkl'),
    ('D:/redata/factor/high_r_std_5m_factor.pkl', 'D:/redata/Neutraliza/high_r_std_5m_nu.pkl'),
    ('D:/redata/factor/hpl_r_std_6m_factor.pkl', 'D:/redata/Neutraliza/hpl_r_std_6m_nu.pkl'),
    ('D:/redata/factor/BP_factor.pkl', 'D:/redata/Neutraliza/BP_nu.pkl'),
    ('D:/redata/factor/total_marketcap_factor.pkl', 'D:/redata/Neutraliza/total_marketcap_nu.pkl'),
    ('D:/redata/factor/cir_mv_factor.pkl', 'D:/redata/Neutraliza/cir_mv_nu.pkl'),
    ('D:/redata/factor/ret_20_factor.pkl', 'D:/redata/Neutraliza/ret_20_nu.pkl'),
    ('D:/redata/factor/bias_5_factor.pkl', 'D:/redata/Neutraliza/bias_5_nu.pkl'),
    ('D:/redata/factor/total_amount_5_factor.pkl', 'D:/redata/Neutraliza/total_amount_5_nu.pkl'),
    ('D:/redata/factor/adjclose_factor.pkl', 'D:/redata/Neutraliza/adjclose_nu.pkl')
]

for factor_path, output_path in factors:
    df = pd.read_pickle(factor_path)
    df = df.replace([np.inf, -np.inf], np.nan)
    df_clean = df * final_universe
    df_clean = df_clean.dropna(how='all', axis=0)
    factor_type = 'ind'
    factor_rank = factor_neutralize(df_clean, factor_type, if_rank=True, risk=risk, risk_cols=risk_cols)
    factor_rank.to_pickle(output_path)

ret_rank = o1o2.rank(axis=1, ascending=True)

factors_rank = [
    ('D:/redata/Neutraliza/cfp_ttm_nu.pkl'),
    ('D:/redata/Neutraliza/EP_nu.pkl'),
    ('D:/redata/Neutraliza/net_profit_Q_yoy_nu.pkl'),
    ('D:/redata/Neutraliza/turnover_std_nu.pkl'),
    ('D:/redata/Neutraliza/mv_nu.pkl'),
    ('D:/redata/Neutraliza/Var120_nu.pkl'),
    ('D:/redata/Neutraliza/hml_r_std_6m_nu.pkl'),
    ('D:/redata/Neutraliza/liquidity_nu.pkl'),
    ('D:/redata/Neutraliza/APB_20_nu.pkl'),
    ('D:/redata/Neutraliza/high_r_std_5m_nu.pkl'),
    ('D:/redata/Neutraliza/hpl_r_std_6m_nu.pkl'),
    ('D:/redata/Neutraliza/BP_nu.pkl'),
    ('D:/redata/Neutraliza/total_marketcap_nu.pkl'),
    ('D:/redata/Neutraliza/cir_mv_nu.pkl'),
    ('D:/redata/Neutraliza/ret_20_nu.pkl'),
    ('D:/redata/Neutraliza/bias_5_nu.pkl'),
    ('D:/redata/Neutraliza/total_amount_5_nu.pkl'),
    ('D:/redata/Neutraliza/adjclose_nu.pkl')
]

for factor_path in factors_rank:
    factor_rank = pd.read_pickle(factor_path)
    IC, ICres, ICIR_120_weight = compute_IC(factor_rank, ret_rank)
    ICIR_120_weight.to_pickle(factor_path.replace('.pkl', '_120_weight.pkl'))

# 相关性检验

In [144]:
comb_mean = (f1+f2+f3+f4+f8+f10+f11+f13)
comb_mean.to_pickle('D:/redata/factor/comb_mean.pkl')

#市值分层
comb_mean_s = comb_mean * small
comb_mean_s.to_pickle('D:/redata/factor/comb_mean_s.pkl')
comb_mean_m = comb_mean * median
comb_mean_m.to_pickle('D:/redata/factor/comb_mean_m.pkl')
comb_mean_sm = comb_mean * sm
comb_mean_sm.to_pickle('D:/redata/factor/comb_mean_sm.pkl')
comb_mean_l = comb_mean * large
comb_mean_l.to_pickle('D:/redata/factor/comb_mean_l.pkl')

#5、10、20日
a = position(comb_mean_sm, 5)
a.to_pickle('D:/redata/factor/comb_mean_sm5d.pkl')
b = position(comb_mean_sm, 10)
b.to_pickle('D:/redata/factor/comb_mean_sm10d.pkl')
d = position(comb_mean_sm, 20)
d.to_pickle('D:/redata/factor/comb_mean_sm20d.pkl')

# 选股

In [146]:
df = pd.read_pickle('D:/redata/factor/comb_mean.pkl')###读入因子值
df = df.replace(np.inf,np.nan).replace(-np.inf,np.nan)
#剔除 新股 st halt
df_clean = (df * final_universe)
df_clean = df_clean.dropna(how='all',axis=0)
factor_type = 'hist'  # 'hist' 因子原值， 'ind' 对因子做行业中性
factor_rank = factor_neutralize(df_clean,factor_type,if_rank=True,risk=risk,risk_cols=['logcmv','retsum120']) 

select_num_stocks(n=10)

ticker,000798,002051,300684,301089,301179,603668,605133,688270,688313,688697
date,,,,,,,,,,
2023-06-02,1,1,1,1,1,1,1,1,1,1
